### Cleaning Bank Marketing Campaign - DataCamp Task 
### 
Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

**Exploring Data**

In [30]:
import pandas as pd
import numpy as np

# Read data 
df = pd.read_csv("bank_marketing.csv")

# Display first few rows
print("First 5 rows of the dataset:")
print(df.head())

# Additional useful information
print("\n" + "="*80)
print("\nDataset Shape:")
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")

print("\n" + "="*80)
print("\nColumn Names and Data Types:")
print(df.dtypes)

print("\n" + "="*80)
print("\nColumn Names List:")
print(df.columns.tolist())

print("\n" + "="*80)
print("\nBasic Info:")
print(df.info())

print("\n" + "="*80)
print("\nCheck for missing values:")
print(df.isnull().sum())


First 5 rows of the dataset:
   client_id  age  ... euribor_three_months campaign_outcome
0          0   56  ...                4.857               no
1          1   57  ...                4.857               no
2          2   37  ...                4.857               no
3          3   40  ...                4.857               no
4          4   56  ...                4.857               no

[5 rows x 16 columns]


Dataset Shape:
Rows: 41188, Columns: 16


Column Names and Data Types:
client_id                       int64
age                             int64
job                            object
marital                        object
education                      object
credit_default                 object
mortgage                       object
month                          object
day                             int64
contact_duration                int64
number_contacts                 int64
previous_campaign_contacts      int64
previous_outcome               object
cons_price_idx 

**Client Table **

In [31]:
# Create client DataFrame - selecting only the columns we need
client = df[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'mortgage']].copy()
print("\nClient DataFrame Info:")
print(client.info())

# CLEANING & TRANSFORMATIONS  
## Job Column
client['job'] = client['job'].str.replace('.', '_', regex=False)

## Education Column
client['education'] = client['education'].str.replace('.', '_', regex=False)
client['education'] = client['education'].replace('unknown', np.nan)

## Credit Default Column
client['credit_default'] = (client['credit_default'] == 'yes').astype(bool)

## Mortgage Column
client['mortgage'] = (client['mortgage'] == 'yes').astype(bool)

# VERIFICATION 
print("AFTER Cleaning:")
print(client.head(10))
print(f"Education: {client['education'].unique()}")
print(f"Credit_default: {client['credit_default'].unique()}")
print(f"Mortgage: {client['mortgage'].unique()}")

# SAVE TO CSV (without index)
client.to_csv('client.csv', index=False)
print("\n client.csv saved successfully!")


Client DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   client_id       41188 non-null  int64 
 1   age             41188 non-null  int64 
 2   job             41188 non-null  object
 3   marital         41188 non-null  object
 4   education       41188 non-null  object
 5   credit_default  41188 non-null  object
 6   mortgage        41188 non-null  object
dtypes: int64(2), object(5)
memory usage: 2.2+ MB
None
AFTER Cleaning:
   client_id  age          job  ...            education credit_default  mortgage
0          0   56    housemaid  ...             basic_4y          False     False
1          1   57     services  ...          high_school          False     False
2          2   37     services  ...          high_school          False      True
3          3   40       admin_  ...             basic_6y          False  

**Campaign Table**

In [32]:
# Create campaign DataFrame - selecting only the columns we need
campaign = df[['client_id', 'number_contacts', 'contact_duration', 'previous_campaign_contacts', 
               'previous_outcome', 'campaign_outcome', 'day', 'month']].copy()

print("\nCampaign DataFrame Info:")
print(campaign.info())

print("\nBEFORE Cleaning:")
print(campaign.head(10))
print(f"\nPrevious_outcome unique: {campaign['previous_outcome'].unique()}")
print(f"Campaign_outcome unique: {campaign['campaign_outcome'].unique()}")
print(f"Month unique: {campaign['month'].unique()}")

# CLEANING & TRANSFORMATIONS  

## Previous Outcome Column  
campaign['previous_outcome'] = (campaign['previous_outcome'] == 'success').astype(bool)

## Campaign Outcome Column  
campaign['campaign_outcome'] = (campaign['campaign_outcome'] == 'yes').astype(bool)

## Create year column with value 2022
campaign['year'] = 2022

## Create last_contact_date from day, month, year
campaign['last_contact_date'] = pd.to_datetime(
    campaign['year'].astype(str) + '-' + campaign['month'] + '-' + campaign['day'].astype(str),
    format='%Y-%b-%d'
)

## Drop the temporary columns (day, month, year) 
campaign = campaign.drop(columns=['day', 'month', 'year'])

# VERIFICATION 
print("\n" + "="*80)
print("AFTER Cleaning:")
print(campaign.head(10))
print(f"\nPrevious_outcome unique: {campaign['previous_outcome'].unique()}")
print(f"Campaign_outcome unique: {campaign['campaign_outcome'].unique()}")
print(f"\nLast_contact_date sample:")
print(campaign['last_contact_date'].head(10))
print(f"\nData type of last_contact_date: {campaign['last_contact_date'].dtype}")

print("\n" + "="*80)
print("\nFinal Campaign DataFrame Info:")
print(campaign.info())

# SAVE TO CSV (without index)
campaign.to_csv('campaign.csv', index=False)
print("\n✅ campaign.csv saved successfully!")



Campaign DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   client_id                   41188 non-null  int64 
 1   number_contacts             41188 non-null  int64 
 2   contact_duration            41188 non-null  int64 
 3   previous_campaign_contacts  41188 non-null  int64 
 4   previous_outcome            41188 non-null  object
 5   campaign_outcome            41188 non-null  object
 6   day                         41188 non-null  int64 
 7   month                       41188 non-null  object
dtypes: int64(5), object(3)
memory usage: 2.5+ MB
None

BEFORE Cleaning:
   client_id  number_contacts  contact_duration  ...  campaign_outcome day month
0          0                1               261  ...                no  13   may
1          1                1               149  ...                no 

**Economics Table**

In [33]:
# Create economics DataFrame - selecting only the columns we need
economics = df[['client_id', 'cons_price_idx', 'euribor_three_months']].copy()

print("\nEconomics DataFrame Info:")
print(economics.info())

print("\nBEFORE Cleaning:")
print(economics.head(10))


# VERIFICATION 
print("\n" + "="*80)
print("AFTER Cleaning:")
print(economics.head(10))

print("\n" + "="*80)
print("\nDescriptive Statistics:")
print(economics.describe())

print("\n" + "="*80)
print("\nFinal Economics DataFrame Info:")
print(economics.info())

# SAVE TO CSV (without index)
economics.to_csv('economics.csv', index=False)
print("\n economics.csv saved successfully!")


Economics DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   client_id             41188 non-null  int64  
 1   cons_price_idx        41188 non-null  float64
 2   euribor_three_months  41188 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 965.5 KB
None

BEFORE Cleaning:
   client_id  cons_price_idx  euribor_three_months
0          0          93.994                 4.857
1          1          93.994                 4.857
2          2          93.994                 4.857
3          3          93.994                 4.857
4          4          93.994                 4.857
5          5          93.994                 4.857
6          6          93.994                 4.857
7          7          93.994                 4.857
8          8          93.994                 4.857
9          9          93.